<a href="https://colab.research.google.com/github/mmao114/Analysis-of-basic-information-and-reviews-of-ski-resorts/blob/main/pcaski.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 安装 janome 库（确保能够进行日语分词）
!pip install janome


In [ ]:

# 2. 导入必要的库
import os
import pandas as pd
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

# 3. 指定文件夹路径并读取文件
folder_path = '/content/11111'  # 上传的文件夹路径

# 获取文件夹中的所有Excel文件名
file_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.xlsx')]

# 创建一个空的DataFrame来存储所有数据
all_data = pd.DataFrame()

# 读取每个文件的数据并合并，同时为每个文件添加滑雪场名称
for file in file_paths:
    df = pd.read_excel(file)
    df['滑雪场'] = os.path.basename(file).replace('.xlsx', '')  # 用文件名作为滑雪场名称
    all_data = pd.concat([all_data, df], ignore_index=True)

# 4. 检查和处理日期数据（評価の時間列）

# Check if all_data is empty and print its shape
if all_data.empty:
    print("all_data is empty. Check file paths and data loading.")
    print(f"Shape of all_data: {all_data.shape}")

#  Specify the date format - replace '%Y-%m-%d %H:%M:%S' with the actual format in your data
all_data = all_data[pd.to_datetime(all_data['評価の時間'], errors='coerce', format='%Y-%m-%d %H:%M:%S').notnull()]
all_data['評価の時間'] = pd.to_datetime(all_data['評価の時間'], errors='coerce', format='%Y-%m-%d %H:%M:%S')

# 根据2020年1月1日划分数据
before_2020 = all_data[all_data['評価の時間'] < '2020-01-01']
after_2020 = all_data[all_data['評価の時間'] >= '2020-01-01']

# 5. 对国籍进行独热编码
all_data = pd.get_dummies(all_data, columns=['国籍'], drop_first=True)

# 查看独热编码后的数据
print("独热编码后的数据：")
print(all_data.head())

# 6. 初始化 janome 分词器
tokenizer = Tokenizer()

# 检查 '口コミ' 列中的空值，并将其填充为空字符串，确保所有数据都是字符串类型
all_data['口コミ'] = all_data['口コミ'].fillna('')  # 将NaN填充为空字符串
all_data['口コミ'] = all_data['口コミ'].astype(str)

# 定义一个分词函数，将口コミ进行分词
def tokenize_text(text):
    tokens = tokenizer.tokenize(text, wakati=True)  # wakati=True 只提取词
    return ' '.join(tokens)

# 对口コミ列进行分词
all_data['口コミ_分词'] = all_data['口コミ'].apply(tokenize_text)

# 查看分词后的口コミ
print("分词后的口コミ：")
print(all_data[['口コミ', '口コミ_分词']].head())

# 7. 使用TfidfVectorizer将分词后的口コミ进行TF-IDF转换
tfidf = TfidfVectorizer(max_features=500, token_pattern=r'(?u)\b\w+\b', stop_words=None)

# 对分词后的口コミ列进行TF-IDF转换
print(all_data['口コミ_分词'].isnull().sum())  # Check for NaN values
   print(all_data[all_data['口コミ_分词'] == ''].shape)  # Check for empty strings

# 将结果转换为DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out())

# 查看转换后的文本特征
print("TF-IDF转换后的特征：")
print(tfidf_df.head())

# 8. 合并所有特征（去掉不需要的列）
all_features = pd.concat([all_data.drop(['口コミ', '口コミ_分词', '評価の時間'], axis=1), tfidf_df], axis=1)

# 9. 使用StandardScaler对数据进行标准化
scaler = StandardScaler()
all_features_scaled = scaler.fit_transform(all_features)

# 查看标准化后的数据
print("标准化后的数据：")
print(all_features_scaled[:5])


IndentationError: unexpected indent (<ipython-input-27-4c7d0e56ddd5>, line 69)

In [ ]:
import pandas as pd
import os

# 假设 numerical_features 包含数值型特征的原始数据
# 创建标准化后的 DataFrame
all_features_scaled_df = pd.DataFrame(all_features_scaled, columns=numerical_features.columns[:all_features_scaled.shape[1]])

# 检查目录是否存在，如果不存在则创建
if not os.path.exists('/mnt/data'):
    os.makedirs('/mnt/data')

# 将标准化后的数据保存为 Excel 文件
all_features_scaled_df.to_excel('/mnt/data/all_features_scaled.xlsx', index=False)

# 提示文件已保存成功
print("文件已成功导出为 Excel 格式：all_features_scaled.xlsx")

文件已成功导出为 Excel 格式：all_features_scaled.xlsx


In [ ]:
from google.colab import files

# 定义要下载的文件路径
file_path = '/mnt/data/all_features_scaled.xlsx'  # 将此路径替换为实际文件路径

# 下载文件
files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 导入必要的库
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 假设你已经在 Google Colab 中上传了文件，并且已经读取数据
# 读取文件
file_name = '/content/all_features_scaled (1).xlsx'  # 确保使用正确的文件路径
data = pd.read_excel(file_name)

# 检查数据是否有 NaN 或无效值
print("NaN 值的数量:", data.isnull().sum().sum())

# 如果有 NaN 值，使用均值填充 NaN
data.fillna(data.mean(), inplace=True)

# 执行 PCA 分析
pca = PCA(n_components=10)  # 保留10个主成分
pca_result = pca.fit_transform(data)

# 查看每个主成分的解释方差比例
explained_variance_ratio = pca.explained_variance_ratio_
print("各主成分的解释方差比例：", explained_variance_ratio)

# 累积解释方差比例
cumulative_variance = explained_variance_ratio.cumsum()
print("累积解释方差比例：", cumulative_variance)

# 设定字体文件的路径（假设你的字体文件名是W4.ttc）
font_path = '/content/W4.ttc'  # 上传字体后的路径

# 使用FontProperties加载字体
prop = fm.FontProperties(fname=font_path)


# 绘制累积解释方差比例图
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--')
plt.title('累積説明分散割合', fontsize=14, fontproperties=prop)
plt.xlabel('主成分数', fontsize=12, fontproperties=prop)
plt.ylabel('累積説明分散割合', fontsize=12, fontproperties=prop)
plt.grid(True)
plt.show()

# 创建主成分得分的 DataFrame
pca_scores = pd.DataFrame(pca_result, columns=[f'主成分{i+1}' for i in range(pca_result.shape[1])])

# 查看主成分得分的前几行
print(pca_scores.head())


# 绘制图形
plt.figure(figsize=(8, 6))
plt.scatter(pca_scores['主成分1'], pca_scores['主成分2'], alpha=0.5)
plt.title('主成分1と主成分2の散布図', fontsize=14, fontproperties=prop)
plt.xlabel('主成分1', fontsize=12, fontproperties=prop)
plt.ylabel('主成分2', fontsize=12, fontproperties=prop)
plt.grid(True)
plt.show()


NaN 值的数量: 182000
各主成分的解释方差比例： [0.02285569 0.02125397 0.01905398 0.01802571 0.01734457 0.01591181
 0.01518575 0.01462438 0.0134632  0.01336213]
累积解释方差比例： [0.02285569 0.04410966 0.06316364 0.08118935 0.09853392 0.11444573
 0.12963148 0.14425586 0.15771906 0.17108119]


FileNotFoundError: [Errno 2] No such file or directory: '/content/W4.ttc'

<Figure size 800x600 with 1 Axes>

           主成分1          主成分2          主成分3          主成分4          主成分5  \
0 -3.604531e-01 -2.815878e-01 -5.685525e-01  2.920928e-01 -1.706677e-01   
1 -3.435693e-01 -2.923489e-01 -5.978270e-01  2.673492e-01 -1.183255e-01   
2  2.508449e-01  1.341607e-01  3.246701e-01  6.041906e-01  1.881458e-01   
3  1.572454e-15 -1.134348e-15 -3.058543e-15 -1.855153e-15  8.280832e-15   
4 -7.035330e-16  5.892653e-16 -2.025065e-15  2.016596e-15  1.585203e-15   

           主成分6          主成分7          主成分8          主成分9         主成分10  
0 -2.781926e-01 -5.192148e-02 -2.035100e-01 -3.538220e-01 -2.030520e-01  
1 -2.706620e-01 -6.347112e-02 -2.425582e-01 -3.474068e-01 -2.859741e-01  
2  4.280993e-02  7.100389e-04 -2.170902e-01  1.079011e+00 -2.761162e-01  
3 -3.733451e-16  9.027229e-15  3.006488e-15 -4.028631e-15  2.613482e-15  
4  2.473576e-15 -5.759849e-16  1.111971e-14  5.349352e-16 -4.095299e-15  


FileNotFoundError: [Errno 2] No such file or directory: '/content/W4.ttc'

<Figure size 800x600 with 1 Axes>